Using Kaggle API to access dataset

In [ ]:
!pip install kaggle
!pip install datasets
!pip install transformers
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [ ]:
!mkdir ~/.kaggle

In [ ]:
cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d samanazhar/multilingual-sentiment-analysis

100% 1.81G/1.81G [01:02<00:00, 31.3MB/s]
100% 1.81G/1.81G [01:02<00:00, 31.1MB/s]


In [ ]:
!unzip /content/multilingual-sentiment-analysis.zip

Archive:  /content/multilingual-sentiment-analysis.zip
  inflating: PMLN_predicted_tweets.csv  
  inflating: PPP_predicted_tweets.csv  
  inflating: PTI_predicted_tweets.csv  
  inflating: Scraped_Tweets/PMLN_Complete_Dataset.csv  
  inflating: Scraped_Tweets/PPP_Complete_Dataset.csv  
  inflating: Scraped_Tweets/PTI_Complete_Dataset.csv  


Importing Libraries

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
import datasets
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow
import transformers
import tensorflow as tf
from tensorflow.keras import layers
import re

Importing Datasets

In [ ]:
ds=pd.read_csv('/content/PMLN_predicted_tweets.csv')
ds1=pd.read_csv('/content/PPP_predicted_tweets.csv')
ds2=pd.read_csv('/content/PTI_predicted_tweets.csv')

In [ ]:
ds=ds[ds['language']=='in']
ds1=ds1[ds1['language']=='in']
ds2=ds2[ds2['language']=='in']

Preprocessing Data

In [ ]:
ds=ds.dropna()
ds1=ds1.dropna()
ds2=ds2.dropna()
ds1 = ds1.drop(ds1[ds1['preprocessed_tweet'].apply(lambda x: isinstance(x, float))].index)
ds1 = ds.drop(ds[ds['preprocessed_tweet'].apply(lambda x: isinstance(x, float))].index)
ds1 = ds2.drop(ds2[ds2['preprocessed_tweet'].apply(lambda x: isinstance(x, float))].index)
ds.reset_index(drop=True, inplace=True)
ds1.reset_index(drop=True, inplace=True)
ds2.reset_index(drop=True, inplace=True)

In [ ]:
X=pd.concat([ds['preprocessed_tweet'], ds1['preprocessed_tweet'],ds2['preprocessed_tweet']], ignore_index=True)
y= pd.concat([ds['sentiment'], ds1['sentiment'],ds2['sentiment']], ignore_index=True)
dataset = list(zip(X, y))
X, y = zip(*dataset)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=10,shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=21,shuffle=True)

In [ ]:
def pre_process(ds):
    corpus = []
    for i in range(len(ds)):
        if isinstance(ds[i], str):
            review = re.sub('[^a-zA-Z]', ' ', ds[i])
            review = review.split(' ')
            review = [word for word in review if word != '']
            corpus.append(review)
    return corpus


In [ ]:
x_train=pre_process(x_train)
x_val=pre_process(x_val)
x_test=pre_process(x_test)
tokenizer=Tokenizer()
tokenizer.fit_on_texts(X)
x_train = tokenizer.texts_to_sequences(x_train)
x_val = tokenizer.texts_to_sequences(x_val)
x_test = tokenizer.texts_to_sequences(x_test)
max_len=len(max(X,key=len))
x_train=pad_sequences(x_train, maxlen=max_len, padding='post', truncating='post')
x_test=pad_sequences(x_test, maxlen=max_len, padding='post', truncating='post')
x_val=pad_sequences(x_val, maxlen=max_len, padding='post', truncating='post')

In [ ]:
vocab_size=tokenizer.word_index

Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
model = LogisticRegression()

# Train the model
model.fit(x_train, y_train)

# Make predictions on the test data
y_pred = model.predict(x_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy: ", accuracy)
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", report)

Accuracy:  0.6474976891588539
Confusion Matrix:
 [[1813 4253]
 [1086 7994]]
Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.30      0.40      6066
           1       0.65      0.88      0.75      9080

    accuracy                           0.65     15146
   macro avg       0.64      0.59      0.58     15146
weighted avg       0.64      0.65      0.61     15146



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test, y_pred)

12.705471110821104

Random Forest Classification Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=500, random_state=42)
# Train the model
model.fit(x_train, y_train)

# Make predictions on the test data
y_pred = model.predict(x_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy: ", accuracy)
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", report)

Accuracy:  0.895682028258286
Confusion Matrix:
 [[4815 1251]
 [ 329 8751]]
Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.79      0.86      6066
           1       0.87      0.96      0.92      9080

    accuracy                           0.90     15146
   macro avg       0.91      0.88      0.89     15146
weighted avg       0.90      0.90      0.89     15146



In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test, y_pred)

3.760000815714057